## Secret PK loading

In [ ]:
import os
from secure_key_utils import get_web3_account
from getpass import getpass

ENCRYPTED_PK = os.environ.get("ENCRYPTED_PK")  # encrypted blob
deployer = get_web3_account(ENCRYPTED_PK, getpass())  # decrypts with local pass + keychain

print(f"Deployer address: {deployer.address}")

# 1. Initialization

### i) Configure RPCs & deployer

In [ ]:
from dotenv import load_dotenv
import os
import boa
from web3 import Web3
import logging
import subprocess
import time

from ABIs import createX_abi

logging.basicConfig(level=logging.INFO, format="%(asctime)s -  %(levelname)s - %(message)s")

load_dotenv()
main_chain = "ethereum"

DRPC_API_KEY = os.environ.get("DRPC_API_KEY")
ANKR_API_KEY = os.environ.get("ANKR_API_KEY")

RPCs = {
    # ankr everywhere (reliable for deployments, drpc as fallback)
    "ethereum": "https://rpc.ankr.com/eth/" + ANKR_API_KEY,
    "arbitrum": "https://rpc.ankr.com/arbitrum/" + ANKR_API_KEY,
    "polygon": "https://rpc.ankr.com/polygon/" + ANKR_API_KEY,
    "bsc": "https://rpc.ankr.com/bsc/" + ANKR_API_KEY,
    "base": "https://rpc.ankr.com/base/" + ANKR_API_KEY,
    "avalanche": "https://rpc.ankr.com/avalanche/" + ANKR_API_KEY,
    "optimism": "https://rpc.ankr.com/optimism/" + ANKR_API_KEY,
    "fraxtal": "https://lb.drpc.org/ogrpc?network=fraxtal&dkey=" + DRPC_API_KEY,
    "gnosis": "https://rpc.ankr.com/gnosis/" + ANKR_API_KEY,
    "sonic": "https://rpc.ankr.com/sonic_mainnet/" + ANKR_API_KEY,
    "ink": "https://lb.drpc.org/ogrpc?network=ink&dkey=" + DRPC_API_KEY,
    "mp1": "https://rpc.ankr.com/corn_maizenet/" + ANKR_API_KEY,  # corn
    # "mantle": "https://rpc.ankr.com/mantle/" + ANKR_API_KEY, # also weird opcode issue
    # "fantom": "https://lb.drpc.org/ogrpc?network=fantom&dkey="+DRPC_API_KEY, # Must redeploy later with another evm
    # "xlayer": "https://rpc.ankr.com/xlayer/"+ANKR_API_KEY,  # no mcopy
    # "taiko": "https://lb.drpc.org/ogrpc?network=taiko&dkey="+DRPC_API_KEY, # no push0
}

createX_address = "0xba5Ed099633D3B313e4D5F7bdc1305d3c28ba5Ed"

state_dict = {}


for key in RPCs.keys():
    state_dict[key] = {}
    state_dict[key]["rpc"] = RPCs[key]
    boa.set_network_env(RPCs[key])
    boa.env.add_account(deployer)
    state_dict[key]["boa"] = boa.env  # oops
    state_dict[key]["w3"] = Web3(Web3.HTTPProvider(RPCs[key]))
    state_dict[key]["createx"] = boa.loads_abi(createX_abi).at(createX_address)

for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        logging.info(f"Working with {boa.env.eoa} on {key}, id {boa.env.evm.patch.chain_id}")
        logging.info(
            f"Chain balance is {state_dict[key]['w3'].eth.get_balance(boa.env.eoa)/1e18 :.6f} ETH"
        )
        logging.info(f"CreateX test: {state_dict[key]['createx'].computeCreate2Address(b'',b'')}")

### ii) Parse LZ deployments data (libs and endpoints)

In [ ]:
from LZDeployments import LZDeployments

lz = LZDeployments()
for chain in state_dict:
    metadata = lz.get_chain_metadata(chain)["metadata"]
    dvn_data = lz.get_chain_metadata(chain)

    # Update state dict
    state_dict[chain].update(
        {
            "eid": metadata["eid"],
            "endpoint": metadata["endpointV2"],
            "send_lib": metadata.get("sendUln302", "unavailable"),
            "receive_lib": metadata.get("receiveUln302", "unavailable"),
            "read_lib": metadata.get("readLib1002", "unavailable"),
            "dvns": dvn_data["dvns"],
            "executor": metadata.get("executor", "0x0000000000000000000000000000000000000000"),
        }
    )

    for field in ["send_lib", "receive_lib", "read_lib", "endpoint", "executor"]:
        if state_dict[chain][field] != "unavailable":
            state_dict[chain][field] = Web3().to_checksum_address(
                state_dict[chain][field]
            )  # Print info

    # Print info
    logging.info(f"LZ details for {chain}:")
    logging.info(f"Chain eID: {metadata['eid']}\nEndpoint address: {metadata['endpointV2']}")
    logging.info(f"DVNs: {len(dvn_data['dvns'])}, Read DVNs: {len(dvn_data['dvns_lzread'])}")
    logging.info(
        f"Send lib: {state_dict[chain]['send_lib']}\n"
        f"Receive lib: {state_dict[chain]['receive_lib']}\n"
        f"Read lib: {state_dict[chain]['read_lib']}\n"
        f"Executor: {state_dict[chain]['executor']}\n---"
    )

# 2. Contracts deployment

In [ ]:
# populate previously deployed addresses
state_dict["ethereum"]["oracle"] = "0xB10CFACE40490D798770FEdd104e0a013eD308a6"
state_dict["arbitrum"]["oracle"] = "0xb10cface0f31830b780C453031d8E803b442e0A4"
state_dict["polygon"]["oracle"] = "0xb10cface0f31830b780C453031d8E803b442e0A4"
state_dict["bsc"]["oracle"] = "0xb10cface0f31830b780C453031d8E803b442e0A4"
state_dict["base"]["oracle"] = "0xb10cface0f31830b780C453031d8E803b442e0A4"
state_dict["avalanche"]["oracle"] = "0xb10cface0f31830b780C453031d8E803b442e0A4"
state_dict["optimism"]["oracle"] = "0xb10cface0f31830b780C453031d8E803b442e0A4"
state_dict["fraxtal"]["oracle"] = "0xb10cface0f31830b780C453031d8E803b442e0A4"
state_dict["gnosis"]["oracle"] = "0xb10cface0f31830b780C453031d8E803b442e0A4"
state_dict["sonic"]["oracle"] = "0xb10cface0f31830b780C453031d8E803b442e0A4"
state_dict["ink"]["oracle"] = "0xb10cface0f31830b780C453031d8E803b442e0A4"
state_dict["mp1"]["oracle"] = "0xb10cface0f31830b780C453031d8E803b442e0A4"
# state_dict["fantom"]["oracle"] = "0xB10CFACE7d5aa30ea632e78C660c7ACE41E3e47e"
# state_dict["xlayer"]["oracle"] = "0xB10CFACE7d5aa30ea632e78C660c7ACE41E3e47e"

for key in state_dict:
    pre_deployed = state_dict[key].get("oracle", None)
    if hasattr(pre_deployed, "address"):
        pre_deployed = pre_deployed.address
    if pre_deployed:
        logging.info(f"View/oracle deployed at {pre_deployed} on {key}")
    else:
        logging.info(f"No view/oracle deployed on {key}")

### i) Deploy main view contract and block oracles 

In [ ]:
# On mainnet viewer will tell us ground truth block data
# On other chains block oracles are consensus of various messengers (LZ in this script)

salt_view = bytes.fromhex("b101b2b0aa02b7167d238b98dc1b0b0404a760e80058c81d87d8bfb3019a17cf")
salt_oracle = bytes.fromhex("b101b2b0aa02b7167d238b98dc1b0b0404a760e8003599f2b3707827003ec333")

# salt_oracle = bytes.fromhex("b101b2b0aa02b7167d238b98dc1b0b0404a760e800f893d170dbd69b0256c8ed") # paris compiled


for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        pre_deployed = state_dict[key].get("oracle", None)

        if hasattr(pre_deployed, "address"):
            # already a boa contract
            continue

        if key == main_chain:
            contract_deployer = boa.load_partial("../contracts/MainnetBlockView.vy")
            deploycode = contract_deployer.compiler_data.bytecode
            if pre_deployed is None:
                address = state_dict[key]["createx"].deployCreate2(salt_view, deploycode)
                logging.info(f"Block view deployed at {address} on {key}")
            else:
                address = pre_deployed
                logging.info(f"Pre-deployed at {address} on {key}")
        else:
            # deploying block oracle on other chains
            contract_deployer = boa.load_partial("../contracts/BlockOracle.vy")
            bytecode = contract_deployer.compiler_data.bytecode
            args = boa.util.abi.abi_encode("(address)", (boa.env.eoa,))
            deploycode = bytecode + args
            if pre_deployed is None:
                address = state_dict[key]["createx"].deployCreate2(salt_oracle, deploycode)
                time.sleep(1)
                logging.info(f"Block oracle deployed at {address} on {key}")
            else:
                address = pre_deployed
                logging.info(f"Pre-deployed at {address} on {key}")
        state_dict[key]["oracle"] = contract_deployer.at(address)

In [ ]:
# optional (at redeploys of messengers) - check committers! (and remove if needed)
for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        if key == main_chain:
            continue
        oracle = state_dict[key]["oracle"]
        committers = oracle.get_all_committers()
        logging.info(f"Committers on {key}: {committers}")

In [ ]:
# populate previously deployed addresses
state_dict["arbitrum"]["block_relay"] = "0xfacefeedcc1a5FDdCa97a20511e6641a5c44370F"
state_dict["polygon"]["block_relay"] = "0xfacefeedcc1a5FDdCa97a20511e6641a5c44370F"
state_dict["bsc"]["block_relay"] = "0xfacefeedcc1a5FDdCa97a20511e6641a5c44370F"
state_dict["base"]["block_relay"] = "0xfacefeedcc1a5FDdCa97a20511e6641a5c44370F"
state_dict["avalanche"]["block_relay"] = "0xfacefeedcc1a5FDdCa97a20511e6641a5c44370F"
state_dict["optimism"]["block_relay"] = "0xfacefeedcc1a5FDdCa97a20511e6641a5c44370F"
state_dict["fraxtal"]["block_relay"] = "0xfacefeedcc1a5FDdCa97a20511e6641a5c44370F"
state_dict["gnosis"]["block_relay"] = "0xfacefeedcc1a5FDdCa97a20511e6641a5c44370F"
state_dict["sonic"]["block_relay"] = "0xfacefeedcc1a5FDdCa97a20511e6641a5c44370F"
state_dict["ink"]["block_relay"] = "0xfacefeedcc1a5FDdCa97a20511e6641a5c44370F"
state_dict["mp1"]["block_relay"] = "0xfacefeedcc1a5FDdCa97a20511e6641a5c44370F"
# state_dict["fantom"]["block_relay"] = "0xfacefEed9989e422241f50B4D277aB0ba0Cc7EA2"
# state_dict["xlayer"]["block_relay"] = "0xfacefEed9989e422241f50B4D277aB0ba0Cc7EA2"
for key in state_dict:
    pre_deployed = state_dict[key].get("block_relay", None)
    if hasattr(pre_deployed, "address"):
        pre_deployed = pre_deployed.address
    if pre_deployed:
        logging.info(f"Relayer deployed at {pre_deployed} on {key}")
    else:
        logging.info(f"No relayer deployed on {key}")

In [ ]:
# Deploy LZBlockRelay on each chain
# This contract is used to send and receive messages via LZ, including LZRead and chained broadcasting
salt_relay = bytes.fromhex("b101b2b0aa02b7167d238b98dc1b0b0404a760e8008ea45ef4f3341d02477927")

# salt_relay = bytes.fromhex("b101b2b0aa02b7167d238b98dc1b0b0404a760e800ea8baffdfa0037027e2112") # paris compiled

for key in state_dict:
    logging.info(f"Switching to {key}...")
    if key == main_chain:
        continue

    pre_deployed = state_dict[key].get("block_relay", None)
    if hasattr(pre_deployed, "address"):
        # already a boa contract
        continue

    with boa.swap_env(state_dict[key]["boa"]):
        contract_deployer = boa.load_partial("../contracts/messengers/LZBlockRelay.vy")
        bytecode = contract_deployer.compiler_data.bytecode
        args = boa.util.abi.abi_encode("(address)", (boa.env.eoa,))
        deploycode = bytecode + args
        if pre_deployed is None:
            address = state_dict[key]["createx"].deployCreate2(salt_relay, deploycode)
            time.sleep(1)
            logging.info(f"Messenger deployed at {address} on {key}")
        else:
            address = pre_deployed
            logging.info(f"Pre-deployed at {address} on {key}")

        contract = contract_deployer.at(address)
        state_dict[key]["block_relay"] = contract

# 3. Contracts configuration

## i) Oracles configuration

#### Initialize with all deployed peer ids and read libs

In [ ]:
# Prepare initialization parameters for block relay contracts
for key in state_dict.keys():
    if key == main_chain:
        continue

    # Basic parameters
    endpoint = state_dict[key]["endpoint"]
    default_gas_limit = 300_000
    read_channel = 4294967295 if state_dict[key]["read_lib"] != "unavailable" else 0

    # Use dict to manage unique peer relationships
    peers_dict = {}

    # 1. If read-enabled:
    # - add self as read peer
    # - add all other chains as peers for broadcasting
    if state_dict[key]["read_lib"] != "unavailable":
        peers_dict[read_channel] = state_dict[key]["block_relay"].address

        for target_key in state_dict.keys():
            if target_key != main_chain and target_key != key:
                peers_dict[state_dict[target_key]["eid"]] = state_dict[target_key][
                    "block_relay"
                ].address

    # 2. For all chains: add read-enabled chains as peers (to receive their broadcasts)
    # duplicates are managed by dict structure
    for source_key in state_dict.keys():
        if (
            source_key != main_chain
            and source_key != key
            and state_dict[source_key]["read_lib"] != "unavailable"
        ):
            peers_dict[state_dict[source_key]["eid"]] = state_dict[source_key][
                "block_relay"
            ].address

    # Convert dict to lists
    peer_eids = list(peers_dict.keys())
    peers = list(peers_dict.values())

    # Prepare lib configuration
    oapps = []
    channels = []
    libs = []
    lib_types = []

    # Add send and receive libs
    # iterate over peers_dict
    for peer_eid, peer_address in peers_dict.items():
        if peer_eid == read_channel or peer_eid == state_dict[key]["eid"]:
            # skip read channel and own eid
            continue
        if state_dict[key]["send_lib"] != "unavailable":
            oapps.append(peer_address)
            channels.append(peer_eid)
            libs.append(state_dict[key]["send_lib"])
            lib_types.append(1)

        if state_dict[key]["receive_lib"] != "unavailable":
            oapps.append(peer_address)
            channels.append(peer_eid)
            libs.append(state_dict[key]["receive_lib"])
            lib_types.append(2)

    # now add read libs explicitly
    if state_dict[key]["read_lib"] != "unavailable":
        oapps.extend([state_dict[key]["block_relay"].address] * 2)
        channels.extend([read_channel, read_channel])
        libs.extend([state_dict[key]["read_lib"], state_dict[key]["read_lib"]])
        lib_types.extend([1, 2])

    logging.info(f"init parameters on {key}")
    logging.info(f"Peer_eids: {peer_eids}")
    logging.info(f"Peers: {peers}")
    logging.info(f"Oapps: {oapps}")
    logging.info(f"Channels: {channels}")
    logging.info(f"Libs: {libs}")
    logging.info(f"Lib types: {lib_types}")
    logging.info(f"Initializing block relay on {key}...")

    # Initialize
    with boa.swap_env(state_dict[key]["boa"]):
        relay_contract = state_dict[key]["block_relay"]
        if not relay_contract.is_initialized():
            relay_contract.initialize(
                endpoint,
                default_gas_limit,
                read_channel,
                peer_eids,
                peers,
                oapps,
                channels,
                libs,
                lib_types,
            )
        else:
            logging.info(f"Already initialized on {key}!")

#### Add block oracles and committer

In [ ]:
for key in state_dict.keys():
    if key == main_chain:
        continue
    with boa.swap_env(state_dict[key]["boa"]):
        relay_contract = state_dict[key]["block_relay"]
        # add block oracles to every relayer
        if relay_contract.block_oracle() == state_dict[key]["oracle"].address:
            logging.info(f"Skipping {key} - already set")
        else:
            relay_contract.set_block_oracle(state_dict[key]["oracle"].address)
            logging.info(f"Set block oracle tx on {key}")

        # now add relayer as committer to block oracle
        oracle_contract = state_dict[key]["oracle"]
        if oracle_contract.is_committer(relay_contract.address):
            logging.info(f"Skipping {key} - already a committer")
        else:
            oracle_contract.add_committer(relay_contract.address, True)
            logging.info(f"Add committer tx on {key}")

        # add mainnet view contract on read chains (as read source)
        if state_dict[key]["read_lib"] != "unavailable":
            if relay_contract.mainnet_block_view() != state_dict[main_chain]["oracle"].address:
                relay_contract.set_read_config(
                    True, state_dict[main_chain]["eid"], state_dict[main_chain]["oracle"].address
                )
                logging.info(f"Set read config on {key}")
            else:
                logging.info(f"Skipping {key} - already set")

## ii) LZ Configuration - removed - all at init

## II. Post-deployment interactions 
## (web3py to simulate real interactions)

### 0. Prepare infra

In [ ]:
from web3.middleware import ExtraDataToPOAMiddleware


def get_vyper_abi(filepath):
    command = ["vyper", filepath, "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


ABI_RELAY = get_vyper_abi("../contracts/messengers/LZBlockRelay.vy")
ABI_ORACLE = get_vyper_abi("../contracts/BlockOracle.vy")
for key in state_dict.keys():
    if key == main_chain:
        continue
    try:
        state_dict[key]["w3"].middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)
    except Exception:
        pass
    try:
        relay_address = state_dict[key]["block_relay"].address
        oracle_address = state_dict[key]["oracle"].address
    except Exception:
        relay_address = state_dict[key]["block_relay"]
        oracle_address = state_dict[key]["oracle"]
    state_dict[key]["block_relay_w3"] = state_dict[key]["w3"].eth.contract(
        address=relay_address, abi=ABI_RELAY
    )
    state_dict[key]["oracle_w3"] = state_dict[key]["w3"].eth.contract(
        address=oracle_address, abi=ABI_ORACLE
    )
account = Web3().eth.account.from_key(deployer.key)


def send_tx_single(w3, func, acc, value=0, gas=0):
    tx = func.build_transaction(
        {
            "from": account.address,
            "nonce": w3.eth.get_transaction_count(account.address),
            "value": value,
        }
    )
    if gas > 0:
        tx["gas"] = gas
    else:
        tx["gas"] = int(w3.eth.estimate_gas(tx))
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return tx_hash


def send_tx(w3, func, acc, value=0, gas=0):
    success = False
    while not success:
        try:
            tx_hash = send_tx_single(w3, func, acc, value, gas)
            success = True
        except Exception as e:
            if (
                "replacement transaction underpriced" in str(e)
                or "nonce too low" in str(e)
                or "could not replace existing tx" in str(e)
            ):
                print(str(e), "Retrying...")
                success = False
                time.sleep(1)
            else:
                raise e
    return tx_hash

### Simple reads without broadcast

In [ ]:
main_block = state_dict[main_chain]["w3"].eth.block_number
print(f"Current block number on {main_chain}: {main_block}")
for key in state_dict.keys():
    if key == main_chain or state_dict[key]["read_lib"] == "unavailable":
        logging.info(f"Skipping {key}")
        continue
    contract_w3 = state_dict[key]["block_relay_w3"]

    # First quote read fee
    fee = contract_w3.functions.quote_read_fee().call()
    logging.info(f"Read fee: {fee} on {key}")

    # Then request read
    func = contract_w3.functions.request_block_hash([], [])
    tx_hash = send_tx(state_dict[key]["w3"], func, account, 3 * fee)
    logging.info(f"Tx: {tx_hash.hex()} on {key}")

In [ ]:
# Check oracle data (wait until lz message propagates)
for key in state_dict.keys():
    if key == main_chain:
        continue
    number = state_dict[key]["oracle_w3"].functions.last_confirmed_block_number().call()
    block_hash = state_dict[key]["oracle_w3"].functions.block_hash(number).call()
    logging.info(f"Last confirmed block on {key}: {number}")
    logging.info(f"Block hash: {block_hash.hex()}")
    try:
        logging.info(f"Number difference: {main_block - number}")
    except Exception:
        pass

### Now separate broadcast

In [ ]:
broadcaster = "base-sepolia"
broadcaster_eid = state_dict[broadcaster]["eid"]
broadcaster_w3 = state_dict[broadcaster]["block_relay_w3"]
receive_eids = [
    state_dict[key]["eid"] for key in state_dict.keys() if key not in [broadcaster, main_chain]
]
broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(receive_eids).call()
broadcast_fees = [fee for fee in broadcast_fees]
logging.info(f"LZSend fees: {broadcast_fees} on {broadcaster}")

# broadcast call
func = broadcaster_w3.functions.broadcast_latest_block(receive_eids, broadcast_fees)
tx_hash = send_tx(state_dict[broadcaster]["w3"], func, account, sum(broadcast_fees))
logging.info(f"Tx: {tx_hash.hex()} on {broadcaster}")

### Now reads with broadcast

In [ ]:
# we can quote read fee now
# let's pick one of the read chains and quote fee
broadcaster = "optimism"
# for broadcaster in ["arbitrum", "optimism", "base"]:
# if state_dict[broadcaster]["read_lib"] == "unavailable" or broadcaster == main_chain:
#     continue
broadcaster_eid = state_dict[broadcaster]["eid"]
broadcaster_w3 = state_dict[broadcaster]["block_relay_w3"]
keys = [key for key in state_dict.keys() if key not in [broadcaster, main_chain]]
receive_eids = []
for key in keys:
    eid = state_dict[key]["eid"]
    print(f"Checking fee for {key}")
    try:
        broadcaster_w3.functions.quote_broadcast_fees([eid]).call()
        receive_eids.append(eid)
    except Exception as e:
        print(f"Error: {e}")
print(list(zip(keys, receive_eids)))
all_peers = broadcaster_w3.functions.get_configured_eids().call()
print(all_peers)
broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(receive_eids).call()
broadcast_fees = [int(fee * 1.1) for fee in broadcast_fees]
logging.info(
    f"LZSend fees: {broadcast_fees} on {broadcaster}. Total: {sum(broadcast_fees)/1e18} ETH"
)

BROADCAST_GAS = len(receive_eids) * 300_000
read_fee_with_broadcast = broadcaster_w3.functions.quote_read_fee(
    0, BROADCAST_GAS, sum(broadcast_fees)
).call()
logging.info(f"LZRead fee with broadcast: {read_fee_with_broadcast} on {broadcaster}")

# # magic broadcast call
main_block = state_dict[main_chain]["w3"].eth.block_number
func = broadcaster_w3.functions.request_block_hash(receive_eids, broadcast_fees, 0, BROADCAST_GAS)
tx_hash = send_tx(state_dict[broadcaster]["w3"], func, account, read_fee_with_broadcast)
logging.info(f"Tx: {tx_hash.hex()} on {broadcaster}")

In [ ]:
# Check oracle data (wait until lz message propagates)
for key in state_dict.keys():
    if key == main_chain:
        continue
    number = state_dict[key]["oracle_w3"].functions.last_confirmed_block_number().call()
    block_hash = state_dict[key]["oracle_w3"].functions.block_hash(number).call()
    logging.info(f"Last confirmed block on {key}: {number}")
    logging.info(f"Block hash: {block_hash.hex()}")
    try:
        logging.info(f"Number difference: {main_block - number}")
    except Exception:
        pass

### Request remote lzread

In [ ]:
# Pick a read-supporting chain to request the read from
read_chain = "arbitrum-sepolia"
requesting_chain = "optimism-sepolia"

# first use read contracts to estimate read and broadcast fees
fee_multiplier = 2
read_contract = state_dict[read_chain]["block_relay_w3"]
read_fee = fee_multiplier * read_contract.functions.quote_read_fee().call()
broadcast_fee = (
    fee_multiplier
    * read_contract.functions.quote_broadcast_fees([state_dict[requesting_chain]["eid"]]).call()[0]
)
logging.info(f"Read fee: {read_fee} on {read_chain}")
logging.info(f"Broadcast fee: {broadcast_fee} on {read_chain}")

# now request remote read
requesting_contract = state_dict[requesting_chain]["block_relay_w3"]
func = requesting_contract.functions.request_remote_read(
    state_dict[read_chain]["eid"], read_fee, broadcast_fee
)
msg_value = 2 * (read_fee + broadcast_fee)
tx_hash = send_tx(state_dict[requesting_chain]["w3"], func, account, msg_value)
logging.info(f"Tx: {tx_hash.hex()} on {requesting_chain}")

## DVN

### Get current DVN config

In [ ]:
from ABIs import lzreadlib_abi
from eth_abi import decode


def decode_dvn_config(hex_data: str, config_type="read", executor=False):
    hex_data = hex_data.replace("0x", "")

    types = (
        ["(uint64,uint8,uint8,uint8,address[],address[])"]
        if config_type != "read"
        else ["(address,uint8,uint8,uint8,address[],address[])"]
    )
    if executor:
        types = ["(uint32,address)"]
    decoded = decode(types, bytes.fromhex(hex_data))
    if executor:
        return {"executor": decoded[0][1]}
    return {
        "confirmations" if config_type != "read" else "executor": decoded[0][0],
        "requiredDVNCount": decoded[0][1],
        "optionalDVNCount": decoded[0][2],
        "optionalDVNThreshold": decoded[0][3],
        "requiredDVNs": [Web3.to_checksum_address(addr) for addr in decoded[0][4]],
        "optionalDVNs": [Web3.to_checksum_address(addr) for addr in decoded[0][5]],
    }


# prepare reverse dict eid -> name
eid_to_name = {state_dict[key]["eid"]: key for key in state_dict.keys()}

CONFIG_TYPE_READ = 1
CONFIG_TYPE_ULN = 2
CONFIG_TYPE_EXECUTOR = 1
# 1. For each chain, for each lib, for each peer - check DVN config
for key in state_dict.keys():
    logging.info(f"Checking DVN config on {key}...")
    if key == main_chain:
        continue
    contract_w3 = state_dict[key]["block_relay_w3"]
    logging.info(f"Relayer contract: {contract_w3.address}")
    configured_eids = contract_w3.functions.get_configured_eids().call()
    logging.info(f"Configured EIDs: {configured_eids}")
    state_dict[key]["current_dvn_config"] = {}
    w3 = state_dict[key]["w3"]
    lib_send = (
        w3.eth.contract(address=state_dict[key]["send_lib"], abi=lzreadlib_abi)
        if state_dict[key]["send_lib"] != "unavailable"
        else None
    )
    lib_receive = (
        w3.eth.contract(address=state_dict[key]["receive_lib"], abi=lzreadlib_abi)
        if state_dict[key]["receive_lib"] != "unavailable"
        else None
    )
    lib_read = (
        w3.eth.contract(address=state_dict[key]["read_lib"], abi=lzreadlib_abi)
        if state_dict[key]["read_lib"] != "unavailable"
        else None
    )
    for eid in configured_eids:
        if eid > 4294965694:
            eid_to_name[eid] = "read_channel"
        logging.info(f"EID: {eid} -> {eid_to_name[eid]}")
        oapp = contract_w3.functions.LZ_PEERS(eid).call()
        state_dict[key]["current_dvn_config"][eid] = {}
        logging.info(f"Peer oapp: {oapp}")
        if lib_send:
            try:
                send_config = lib_send.functions.getConfig(eid, oapp, CONFIG_TYPE_ULN).call()
                send_config = decode_dvn_config(send_config.hex(), "send")
                executor_config = lib_send.functions.getConfig(
                    eid, oapp, CONFIG_TYPE_EXECUTOR
                ).call()
                executor_config = decode_dvn_config(executor_config.hex(), "send", True)
                send_config["executor"] = executor_config["executor"]
            except Exception as e:
                send_config = None
                logging.debug(f"Send lib config: ERR on {key} - {e}")
        if lib_receive:
            try:
                receive_config = lib_receive.functions.getConfig(eid, oapp, CONFIG_TYPE_ULN).call()
                receive_config = decode_dvn_config(receive_config.hex(), "receive")
            except Exception as e:
                receive_config = None
                logging.debug(f"Receive lib config: ERR on {key} - {e}")
        if lib_read:
            try:
                read_config = lib_read.functions.getConfig(eid, oapp, CONFIG_TYPE_READ).call()
                read_config = decode_dvn_config(read_config.hex(), "read")
            except Exception as e:
                read_config = None
                logging.debug(f"Read lib config: ERR on {key} - {e}")
        if eid >= 4294965694:
            send_config = read_config
            # receive_config = read_config
        logging.info(f"Send lib config: {send_config}")
        logging.info(f"Receive lib config: {receive_config}")
        # logging.info(f"Read lib config: {read_config}")
        state_dict[key]["current_dvn_config"][eid]["send"] = send_config
        state_dict[key]["current_dvn_config"][eid]["receive"] = receive_config
        # state_dict[key]["current_dvn_config"][eid]["read"] = read_config
    logging.info("-" * 100)

### Plan desired DVN config

In [ ]:
def inject_curve_dvns(dvns_list, chain_key):
    # Curve DVN addresses per chain
    curve_dvns = {
        "sepolia": "0x3dc328b85d66362d9b5b359789b5bf3997dbf12e",
        "base-sepolia": "0x6310fb9f8efca6b486276c3ba74a4dd508e41e7a",
        "optimism-sepolia": "0x48f830f47d6c98ae323acd3c6cd1cbac8c6f3ce5",
        "arbitrum-sepolia": "0x2076f14b292abdbef7e2871ca93608755fa741a9",
    }

    # Skip if chain not in mapping
    if chain_key not in curve_dvns:
        return dvns_list

    # Create Curve DVN entry
    curve_dvn = {
        "address": curve_dvns[chain_key],
        "version": 2,
        "canonicalName": "Curve Finance",
        "id": "curve",
        "lzReadCompatible": True,
    }

    # Add to list if not already present
    if not any(d["address"].lower() == curve_dvn["address"].lower() for d in dvns_list):
        dvns_list.append(curve_dvn)

    return dvns_list


def checksum(address):
    return Web3.to_checksum_address(address)


def sorted_a(addresses):
    return sorted(addresses, key=lambda x: int(x, 16))


# Build desired DVN configurations
for key in state_dict.keys():
    if key == main_chain:
        continue

    logging.info(f"Analyzing {key}...")
    contract_w3 = state_dict[key]["block_relay_w3"]
    configured_eids = contract_w3.functions.get_configured_eids().call()
    # Initialize if not exists
    if "desired_dvn_config" not in state_dict[key]:
        state_dict[key]["desired_dvn_config"] = {}

    # Get all DVNs including Curve for this chain
    # dvns = inject_curve_dvns(state_dict[key]["dvns"], key)
    dvns = state_dict[key]["dvns"]
    # For each peer relationship (only process sends, receives will be populated)
    for eid in configured_eids:
        if eid > 4294965694:  # Read channel
            if state_dict[key]["read_lib"] != "unavailable":
                # Get read-compatible DVNs
                read_dvns = [
                    dvn
                    for dvn in dvns
                    if dvn.get("lzReadCompatible", False)
                    and dvn["id"] in ["layerzero-labs", "nethermind", "google-cloud"]
                ]
                if read_dvns:
                    state_dict[key]["desired_dvn_config"][eid] = {
                        "send": {
                            "requiredDVNs": sorted_a(
                                [checksum(dvn["address"]) for dvn in read_dvns]
                            ),
                            "optionalDVNs": [],
                            "executor": state_dict[key]["executor"],
                        },
                        "receive": {"requiredDVNs": [], "optionalDVNs": []},
                    }
            continue

        peer_chain = eid_to_name.get(eid)
        if not peer_chain:
            continue

        # logging.info(f"\n{key} -> {peer_chain} relationship:")
        # peer_dvns = inject_curve_dvns(state_dict[peer_chain]["dvns"], peer_chain)
        peer_dvns = state_dict[peer_chain]["dvns"]
        # Get DVNs by ID for both chains
        # Only consider non-READ-compatible DVNs for ULN
        my_dvns_by_id = {
            d["id"]: d["address"]
            for d in dvns
            if not d.get("lzReadCompatible", False)  # or d["id"] == "curve"
        }
        peer_dvns_by_id = {
            d["id"]: d["address"]
            for d in peer_dvns
            if not d.get("lzReadCompatible", False)  # or d["id"] == "curve"
        }

        # Check available required DVNs
        required_ids = []

        # Try to get all three preferred DVNs
        for dvn_id in ["layerzero-labs"]:
            if dvn_id in my_dvns_by_id and dvn_id in peer_dvns_by_id:
                required_ids.append(dvn_id)

        if not required_ids:
            logging.warning(f"No matching DVNs found for {key}->{peer_chain}")
            continue

        optional_ids = []
        for dvn_id in ["nethermind", "google-cloud"]:
            if dvn_id in my_dvns_by_id and dvn_id in peer_dvns_by_id:
                optional_ids.append(dvn_id)

        # Initialize configs if not exist
        if eid not in state_dict[key]["desired_dvn_config"]:
            state_dict[key]["desired_dvn_config"][eid] = {
                "send": {"requiredDVNs": [], "optionalDVNs": []},
                "receive": {"requiredDVNs": [], "optionalDVNs": []},
            }

        if "desired_dvn_config" not in state_dict[peer_chain]:
            state_dict[peer_chain]["desired_dvn_config"] = {}

        if state_dict[key]["eid"] not in state_dict[peer_chain].get("desired_dvn_config", {}):
            state_dict[peer_chain]["desired_dvn_config"][state_dict[key]["eid"]] = {
                "send": {"requiredDVNs": [], "optionalDVNs": []},
                "receive": {"requiredDVNs": [], "optionalDVNs": []},
            }

        # Set send config for this chain (only if it supports lzread, otherwise it never sends)
        if state_dict[key]["read_lib"] != "unavailable":
            state_dict[key]["desired_dvn_config"][eid]["send"] = {
                "requiredDVNs": sorted_a([checksum(my_dvns_by_id[id]) for id in required_ids]),
                "optionalDVNs": sorted_a([checksum(my_dvns_by_id[id]) for id in optional_ids]),
                "executor": state_dict[key]["executor"],
            }

            # Set receive config for peer chain (only if sender has lzread)
            state_dict[peer_chain]["desired_dvn_config"][state_dict[key]["eid"]]["receive"] = {
                "requiredDVNs": sorted_a([checksum(peer_dvns_by_id[id]) for id in required_ids]),
                "optionalDVNs": sorted_a([checksum(peer_dvns_by_id[id]) for id in optional_ids]),
            }
        else:
            # put the current config in place (if it exists)
            logging.info(f"No lzread on {key}, no send/receive config set")


# Print summary
for key in state_dict.keys():
    if key == main_chain:
        continue
    print(f"\n{key} desired configs:")
    for eid, config in state_dict[key].get("desired_dvn_config", {}).items():
        peer = eid_to_name.get(eid, eid)
        print(f"\nPeer {peer}:")
        if config["send"]["requiredDVNs"]:
            print("Send DVNs:")
            print(f"Required: {config['send']['requiredDVNs']}")
            print(f"Optional: {config['send']['optionalDVNs']}")
            print(f"Executor: {config['send']['executor']}")
        if config["receive"]["requiredDVNs"]:
            print("Receive DVNs:")
            print(f"Required: {config['receive']['requiredDVNs']}")
            print(f"Optional: {config['receive']['optionalDVNs']}")

### Compare desired and current configs

In [ ]:
import json

chain = "arbitrum"
eids_desired = list(state_dict[chain]["desired_dvn_config"].keys())
for eid in eids_desired:
    logging.info(f"EID: {eid}, {eid_to_name[eid]}")
    for config_type in ["send", "receive"]:
        try:
            desired_req_dvns = state_dict[chain]["desired_dvn_config"][eid][config_type][
                "requiredDVNs"
            ]
        except Exception:
            desired_req_dvns = None
        try:
            desired_opt_dvns = state_dict[chain]["desired_dvn_config"][eid][config_type][
                "optionalDVNs"
            ]
        except Exception:
            desired_opt_dvns = None
        try:
            current_req_dvns = state_dict[chain]["current_dvn_config"][eid][config_type][
                "requiredDVNs"
            ]
        except Exception:
            current_req_dvns = None
        try:
            current_opt_dvns = state_dict[chain]["current_dvn_config"][eid][config_type][
                "optionalDVNs"
            ]
        except Exception:
            current_opt_dvns = None
        logging.info(
            f"Desired {config_type} DVNs for {eid_to_name[eid]} {eid}: R:{desired_req_dvns} O:{desired_opt_dvns}"
        )
        logging.info(
            f"Current {config_type} DVNs for {eid_to_name[eid]} {eid}: R:{current_req_dvns} O:{current_opt_dvns}"
        )

### Set desired DVN config

In [ ]:
CONFIG_TYPE_ULN = 2
CONFIG_TYPE_READ = 1


def checksum(address):
    return Web3.to_checksum_address(address)


from_chains = list(state_dict.keys())
to_chains = list(state_dict.keys())
to_chains.append("read_channel")

# from_chains = ["arbitrum"]
# Set all configs in one pass
for key in from_chains:
    if key == main_chain:
        continue

    logging.info(f"\nSetting configs for {key}...")
    contract_w3 = state_dict[key]["block_relay_w3"]
    oapp = checksum(contract_w3.address)

    # Get libs
    libs = {
        "send": checksum(state_dict[key]["send_lib"])
        if state_dict[key]["send_lib"] != "unavailable"
        else None,
        "receive": checksum(state_dict[key]["receive_lib"])
        if state_dict[key]["receive_lib"] != "unavailable"
        else None,
        "read": checksum(state_dict[key]["read_lib"])
        if state_dict[key]["read_lib"] != "unavailable"
        else None,
    }
    logging.info(f"Key: {key}, libs: {libs}")
    # For each peer relationship
    for eid, config in state_dict[key]["desired_dvn_config"].items():
        if key != "polygon" and eid_to_name[eid] != "polygon":
            continue
        is_read_channel = eid > 4294965694
        peer_name = "read_channel" if is_read_channel else eid_to_name.get(eid)
        if peer_name not in to_chains:
            continue
        logging.info(f"\nConfiguring {key} <-> {peer_name}")

        # Loop through send and receive configs
        for config_type in ["send", "receive"]:
            if not config[config_type]["requiredDVNs"]:
                continue

            # Skip receive for read channel
            if is_read_channel and config_type == "receive":
                continue

            # # Skip send if no read lib
            # if state_dict[key]["send_lib"] != "unavailable" and config_type == "send":
            #     continue

            # Determine correct lib and config type
            lib = libs["read"] if is_read_channel else libs[config_type]
            if not lib:
                continue

            config_type_enum = CONFIG_TYPE_READ if is_read_channel else CONFIG_TYPE_ULN

            required_dvns = sorted_a(
                [checksum(addr) for addr in config[config_type].get("requiredDVNs", [])]
            )
            optional_dvns = sorted_a(
                [checksum(addr) for addr in config[config_type].get("optionalDVNs", [])]
            )
            optional_threshold = 1 if len(optional_dvns) >= 1 else 0
            executor = config[config_type].get(
                "executor", "0x0000000000000000000000000000000000000000"
            )
            confirmations = 128

            # # reset to defaults
            # required_dvns = []
            # optional_dvns = []
            # optional_threshold = 0
            # confirmations = 0
            executor = "0x0000000000000000000000000000000000000000"

            logging.info(f"Setting {config_type} config on {key} for {peer_name}:")
            logging.info(f"- Required DVNs: {required_dvns}")
            logging.info(f"- Optional DVNs: {optional_dvns}")
            logging.info(f"- Optional threshold: {optional_threshold}")

            try:
                func = contract_w3.functions.set_lz_uln_config(
                    eid,  # peer eid
                    oapp,  # oapp
                    lib,  # lib
                    config_type_enum,  # config type
                    confirmations,  # confirmations
                    required_dvns,  # required DVNs
                    optional_dvns,  # optional DVNs
                    optional_threshold,  # optional threshold
                    executor,
                )
                # # simulate tx by estimating gas
                # tx = func.build_transaction(
                #     {
                #         "from": account.address,
                #     }
                # )
                # gas_estimate = state_dict[key]["w3"].eth.estimate_gas(tx)
                # logging.info(f"Gas estimate: {gas_estimate}")
                tx_hash = send_tx(state_dict[key]["w3"], func, account)
                logging.info(
                    f"Set {'read' if is_read_channel else config_type} config on {key} for {peer_name}: {tx_hash.hex()}"
                )
                time.sleep(3)
            except Exception as e:
                logging.error(f"Failed to set config on {key} for {peer_name}: {str(e)}")
                logging.info(
                    f"eid: {eid}, oapp: {oapp}, lib: {lib}, config_type_enum: {config_type_enum}, confirmations: {confirmations}, required_dvns: {required_dvns}, optional_dvns: {optional_dvns}, optional_threshold: {optional_threshold}, executor: {executor}"
                )

### Block headers

In [ ]:
for chain_key in state_dict.keys():
    if chain_key == main_chain:
        continue
    # boa because we have it initialized
    block_number = state_dict[chain_key]["oracle"].last_confirmed_block_number()
    print(f"Last confirmed block on {chain_key}: {block_number}")
    block_hash = state_dict[chain_key]["oracle"].block_hash(block_number)
    print(f"Block hash: {block_hash.hex()}")
    # block_data = state_dict[chain_key]["oracle"].block_data(block_number)
    hash1, _, root1, number1, timestamp1 = state_dict[chain_key]["oracle"].block_header(
        block_number
    )
    print(f"Block headers: {hash1.hex()}, {root1.hex()}, {number1}, {timestamp1}")
    # if number1==0 and block_number != 0: # empty header
    #     block_data = state_dict[main_chain]["w3"].eth.get_block(block_number, full_transactions=False)
    #     encoded_headers = encode_headers(block_data)
    #     print(f"Encoded headers: {encoded_headers.hex()}")
    #     state_dict[chain_key]["oracle"].submit_block_header(encoded_headers)
    #     # print(block_data)
    #     headers = state_dict[chain_key]["oracle"].block_header(block_number)
    #     print(f"Block headers: {headers[0].hex()}, {headers[1].hex()}, {headers[2].hex()}, {headers[3]}, {headers[4]}")

## Validate abi on scans

In [ ]:
import requests

# Load contract data
with open("../oracle.json", "r") as f:
    oracle_data = json.load(f)
with open("../relay.json", "r") as f:
    relay_data = json.load(f)

# Configuration
ETHERSCAN_API_KEY = os.environ.get("ETHERSCAN_API_KEY")
BASE_URL = "https://api.etherscan.io/v2/api"

contracts = {
    "oracle": {
        "address": "0xb10cface0f31830b780C453031d8E803b442e0A4",
        "data": oracle_data,
        "name": "BlockOracle.vy:Block Oracle",
    },
    "relay": {
        "address": "0xfacefeedcc1a5FDdCa97a20511e6641a5c44370F",
        "data": relay_data,
        "name": "LZBlockRelay.vy:LZ Block Relay",
    },
}


def verify_contract(
    chain_id: int,
    contract_address: str,
    source_code: str,
    contract_name: str,
    constructor_args: str,
):
    # Build URL with query parameters
    params = {
        "apikey": ETHERSCAN_API_KEY,
        "module": "contract",
        "action": "verifysourcecode",
        "codeformat": "vyper-json",
        "sourceCode": source_code,
        "contractaddress": contract_address,
        "contractname": contract_name,
        "compilerversion": "vyper:0.4.0",
        "optimizationUsed": "1",
        "constructorArguments": constructor_args.hex(),
    }
    url = f"{BASE_URL}?chainid={chain_id}"

    # Print URL for debugging (excluding source code)
    debug_params = {**params, "sourceCode": "..."}
    logging.info(f"Sending request to: {url}")
    logging.info(f"With params: {json.dumps(debug_params, indent=2)}")

    try:
        response = requests.post(url, data=params, timeout=10)
        logging.info(f"Response status code: {response.status_code}")
        logging.info(f"Response text: {response.text}")

        return response.json()
    except Exception as e:
        logging.error(f"Request failed: {str(e)}")
        return None


# Encode constructor arguments (deployer address)
constructor_args = boa.util.abi.abi_encode("(address)", (deployer.address,))
# Submit verification for each contract on each chain
for key in state_dict.keys():
    if key == main_chain:
        continue

    chain_id = state_dict[key]["w3"].eth.chain_id
    logging.info(f"\nSubmitting verifications for {key} (chain_id: {chain_id})")

    for contract_name, contract in contracts.items():
        try:
            result = verify_contract(
                chain_id=chain_id,
                contract_address=contract["address"],
                source_code=json.dumps(contract["data"]),
                contract_name=contract["name"],
                constructor_args=constructor_args,
            )

            if result.get("status") == "1":
                logging.info(f"Verification submitted for {contract_name} on {key}")
                logging.info(f"GUID: {result.get('result')}")
            else:
                logging.error(f"Verification failed for {contract_name} on {key}")
                logging.error(f"Error: {result.get('result')}")

            # Add delay to avoid rate limiting
            time.sleep(1)

        except Exception as e:
            logging.error(f"Exception verifying {contract_name} on {key}: {str(e)}")
            continue

    #     break
    # break

In [ ]:
for key in state_dict.keys():
    state_dict[key]["desired_dvn_config"] = {}
    state_dict[key]["current_dvn_config"] = {}

In [ ]:
state_dict[key]["current_dvn_config"][eid]